In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import time
import math

/Users/mustafamuratarat/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
X_train=pd.read_csv('X_train.txt', header=None).as_matrix()
y_train = pd.read_csv('y_train.txt', header=None).as_matrix().ravel()
X_test = pd.read_csv('X_test.txt', header=None).as_matrix()
y_test = pd.read_csv('y_test.txt', header=None).as_matrix().ravel()

In [3]:
def convertDummy(y, labelNum):
    labelNum = tf.constant(labelNum)
    dummy = tf.one_hot(y, labelNum, axis=1)
    sess = tf.Session()
    dummy = sess.run(dummy)
    sess.close()
    return dummy

In [4]:
def length(sequence):
  used = tf.sign(tf.reduce_max(tf.abs(sequence), 2))
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  sess = tf.Session()
  length = sess.run(length)
  sess.close()
  return length

In [5]:
max_timestep = 60
num_classes = 6
split_size = max_timestep

In [6]:
 def split_time(m, split_size):
        r = m.shape[0]
        extend_row_size = np.math.ceil(r / split_size) * split_size - r
        m_p = np.expand_dims(np.pad(m, [(0, extend_row_size), (0, 0)], mode='constant'), axis=0)
        result = m_p.reshape((np.math.ceil(r / split_size), split_size, m.shape[1]))
        return result

In [7]:
Xtrain = split_time(X_train, split_size)
ytrain = split_time(convertDummy(y_train-1, num_classes), split_size)
Xtest = split_time(X_test, split_size)
ytest = split_time(convertDummy(y_test-1, num_classes), split_size)

In [8]:
print(Xtrain.shape)
print(ytrain.shape)
print(Xtest.shape)
print(ytest.shape)

(123, 60, 561)
(123, 60, 6)
(50, 60, 561)
(50, 60, 6)


In [9]:
length(Xtrain) #the last instance has only 32 time steps. We left-pad them with zeroes.

array([60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 32], dtype=int32)

In [10]:
learning_rate = 0.01
num_neurons = 128
num_inputs = 561
num_classes = 6
num_steps=60
num_iterations = 100
num_layers=3

#During training, you can feed any value you want to the keep_prob placeholder (typically 0.5)
train_keep_prob =0.5

seq_length_batch = length(Xtrain)

X = tf.placeholder(tf.float32, [None, num_steps, num_inputs])
y = tf.placeholder(tf.int64, [None, None, num_classes])
seq_length = tf.placeholder(tf.int32, [None])

keep_prob = tf.placeholder_with_default(1.0, shape=())

initializer = tf.random_uniform_initializer(-1, 1)
lstm_cells = [tf.contrib.rnn.LSTMCell(num_units=num_neurons, state_is_tuple=True, forget_bias=1.0, initializer=initializer) for layer in range(num_layers)]
lstm_cells_drop = [tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=keep_prob) for cell in lstm_cells]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(lstm_cells_drop)
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, sequence_length= seq_length, dtype=tf.float32)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    outputs_val, states_val = sess.run([outputs, states], feed_dict={X: Xtrain, seq_length: seq_length_batch, keep_prob: train_keep_prob})

#The outputs holds the outputs of all cells for all sequences in a batch. So if a particular sequence is shorter and
#padded with zeros, the outputs for the last cells will be zero.

#The states holds the last cell state, or equivalently the last non-zero output per sequence

#states variable is a tuple containing one tensor per layer, each representing the final state of that layer's cell
#with shape [batch_size X num_neurons]

#states_val[2] is the tuple for the last layer. It has two arrays in it , one for hidden state h_state, 
#the other for memory state c_state.

#states_val[2][1].shape hidden state has the shape (123, 128), which is [batch_size X num_neurons] 
#that we mentioned above.

#the hidden state of the last layer in states tuple will be the same with the last time-step of outputs 
#for one particular observation.


In [11]:
states_val[2]

LSTMStateTuple(c=array([[ 1.7748687e-01, -7.3981029e-01,  6.5627635e-01, ...,
         3.4709066e-02, -7.2951591e-01, -1.1264236e+00],
       [-9.9907249e-01, -6.0380000e-01, -4.0375732e-02, ...,
        -8.5740954e-01, -1.1775501e+00,  6.5739131e-01],
       [-8.7011558e-01, -4.8667067e-01, -1.6235726e-04, ...,
        -2.8230852e-01,  1.9982469e-01,  7.4888453e-02],
       ...,
       [ 4.2244363e-01, -4.4949746e-01, -1.6434340e+00, ...,
         1.4370871e+00, -2.4305090e-01, -1.2360177e+00],
       [ 6.5276861e-01,  1.7359817e-01,  3.5736096e-01, ...,
        -1.2055391e+00, -1.0554307e+00,  4.0401384e-01],
       [-9.6683675e-01, -7.7438557e-01, -4.2099595e-01, ...,
        -4.4919476e-01,  1.3397702e-02, -6.0400122e-01]], dtype=float32), h=array([[ 1.12709112e-03, -6.26621783e-01,  6.11348404e-03, ...,
         1.48764858e-02, -6.14504516e-01, -5.19650638e-01],
       [-5.54635073e-04, -1.01621663e-04, -1.49023812e-03, ...,
        -3.48323882e-01, -3.46437931e-01,  3.23214442e-0

In [12]:
outputs_val[0][59]

array([ 1.12709112e-03, -6.26621783e-01,  6.11348404e-03,  3.21006179e-02,
        2.04099659e-02,  2.91427528e-03, -6.48961484e-01, -4.91962419e-06,
        2.99773389e-03,  8.89999390e-01, -2.63846293e-03, -2.79121459e-01,
        6.18381761e-02,  1.14702554e-02,  3.86608532e-04,  8.38070452e-01,
        8.94896090e-01,  1.66724235e-01,  7.08523095e-01,  7.65190244e-01,
        3.06091330e-04, -2.90650964e-01,  5.85308969e-02, -4.64785565e-03,
       -6.84897423e-01, -5.07276803e-02,  6.29149601e-02, -3.14832665e-02,
       -3.01975786e-04, -4.39650565e-03,  4.75072831e-01,  1.12810181e-02,
       -1.62384491e-02,  3.01979661e-01,  7.10277185e-02, -1.33077893e-02,
       -8.63725087e-04, -2.18957543e-01,  7.07012713e-01,  1.36237303e-02,
        2.85759587e-02, -1.01447059e-03, -2.35030234e-01, -1.48291024e-03,
       -3.02080065e-01, -2.54049953e-02, -2.60028243e-03, -8.95632207e-02,
       -2.97868103e-02, -2.84402873e-02, -7.64166472e-08,  7.54637599e-01,
        6.94997832e-02,  

In [13]:
states_val[2][1][0]

array([ 1.12709112e-03, -6.26621783e-01,  6.11348404e-03,  3.21006179e-02,
        2.04099659e-02,  2.91427528e-03, -6.48961484e-01, -4.91962419e-06,
        2.99773389e-03,  8.89999390e-01, -2.63846293e-03, -2.79121459e-01,
        6.18381761e-02,  1.14702554e-02,  3.86608532e-04,  8.38070452e-01,
        8.94896090e-01,  1.66724235e-01,  7.08523095e-01,  7.65190244e-01,
        3.06091330e-04, -2.90650964e-01,  5.85308969e-02, -4.64785565e-03,
       -6.84897423e-01, -5.07276803e-02,  6.29149601e-02, -3.14832665e-02,
       -3.01975786e-04, -4.39650565e-03,  4.75072831e-01,  1.12810181e-02,
       -1.62384491e-02,  3.01979661e-01,  7.10277185e-02, -1.33077893e-02,
       -8.63725087e-04, -2.18957543e-01,  7.07012713e-01,  1.36237303e-02,
        2.85759587e-02, -1.01447059e-03, -2.35030234e-01, -1.48291024e-03,
       -3.02080065e-01, -2.54049953e-02, -2.60028243e-03, -8.95632207e-02,
       -2.97868103e-02, -2.84402873e-02, -7.64166472e-08,  7.54637599e-01,
        6.94997832e-02,  